In [1]:
from vasili_helpers import *

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last"

In [18]:
from faiss.contrib.exhaustive_search import (
    exponential_query_iterator,
    knn,
    knn_ground_truth,
    range_ground_truth,
    range_search_max_results,
)

In [19]:
d = 32
index = faiss.index_factory(d, "IVF5,Flat")
index.train(faiss.randn((100, d), 123))

# reference reconstruction
index.add(faiss.randn((100, d), 345))
index.add(faiss.randn((100, d), 678))
ref_recons = index.reconstruct_n(0, 200)

ref_recons.shape

# with lookup
index.reset()
rs = np.random.RandomState(123)
ids = rs.choice(10000, size=200, replace=False)
index.add_with_ids(faiss.randn((100, d), 345), ids[:100])
index.set_direct_map_type(faiss.DirectMap.Hashtable)
index.add_with_ids(faiss.randn((100, d), 678), ids[100:])

# compare
for i in range(0, 200, 13):
    recons = index.reconstruct(int(ids[i]))
    # self.assertTrue(np.all(recons == ref_recons[i]))

# test I/O
buf = faiss.serialize_index(index)
index2 = faiss.deserialize_index(buf)

# compare
for i in range(0, 200, 13):
    recons = index2.reconstruct(int(ids[i]))
    # self.assertTrue(np.all(recons == ref_recons[i]))

# remove
toremove = np.ascontiguousarray(ids[0:200:3])

sel = faiss.IDSelectorArray(50, faiss.swig_ptr(toremove[:50]))

# test both ways of removing elements
nremove = index2.remove_ids(sel)
nremove += index2.remove_ids(toremove[50:])

# self.assertEqual(nremove, len(toremove))

WARNING clustering 100 points to 5 centroids: please provide at least 195 training points


(200, 32)

In [20]:
from vasili_helpers import *

In [21]:
d = 128
xb, xq = create_random_dataset_fast(d=d, nb=1000000, nq=10_000, seed=1234)

dataset shape:
xb.shape=(1000000, 128)
xq.shape=(10000, 128)


In [22]:
xb.shape
xq.shape

(1000000, 128)

(10000, 128)

In [23]:
import numpy as np
import time

index = faiss.index_factory(d, "IVF1000,Flat")
s = time.perf_counter()
index.train(xb)
e =  time.perf_counter()
print(f"train time {(e-s)*1000} ms")

s = time.perf_counter()
index.add(xb)
e =  time.perf_counter()
print(f"add time {(e-s)*1000} ms")

train time 3146.4835869992385 ms
add time 1257.8279219997057 ms


In [24]:
import time
import numpy as np

nq = 1
index.nprobe = 10

# Function to count number of values not equal to -1 in the indices
def count_non_negative_indices(indices):
    return np.sum(indices != -1, axis=0)

### warm up run
# First query (k=10)
k = 100
s = time.perf_counter()
distances, indices = index.search(xq[:100, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

#####



# First query (k=10)
k = 1
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

k = 10
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

k = 100
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

k = 1000
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

k = 2_000
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

k = 5_000
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

k = 10_000
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ k )}")
print(indices.shape)

query 100 time 2.0201439983793534 ms
(avg)num returned: 10000
(100, 100)
query 1 time 0.27494900132296607 ms
(avg)num returned: 1
(1, 1)
query 10 time 0.2872589975595474 ms
(avg)num returned: 10
(1, 10)
query 100 time 0.29419899874483235 ms
(avg)num returned: 100
(1, 100)
query 1000 time 0.5053989989391994 ms
(avg)num returned: 1000
(1, 1000)
query 2000 time 0.5737089995818678 ms
(avg)num returned: 2000
(1, 2000)
query 5000 time 1.0209870015387423 ms
(avg)num returned: 5000
(1, 5000)
query 10000 time 1.38815600075759 ms
(avg)num returned: 10000
(1, 10000)


In [25]:
import numpy as np

In [28]:
for step in range(1):
    np.arange(100) + step * 100
    np.arange(100) + step * 100 + 500

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

array([500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512,
       513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525,
       526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538,
       539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551,
       552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564,
       565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577,
       578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590,
       591, 592, 593, 594, 595, 596, 597, 598, 599])

Brute Force:

In [33]:
d = 128
xb, xq = create_random_dataset_fast(d=d, nb=1000000, nq=10_000, seed=1234)

dataset shape:
xb.shape=(1000000, 128)
xq.shape=(10000, 128)


In [41]:
faiss.omp_set_num_threads(1)

In [56]:
from faiss.contrib.exhaustive_search import knn_ground_truth
# index = faiss.index_factory(d, "Flat") 
s = time.perf_counter()
faiss.knn(
    xq[0:1,:],
    xb,
    k=nb,
)
e =  time.perf_counter()
print(f"search time {(e-s)*1000} ms")

search time 136.3524159969529 ms


In [58]:
import numpy as np
import time

xb.shape
nb, d = xb.shape

index = faiss.index_factory(d, "Flat") 
s = time.perf_counter()
index.train(xb)
e =  time.perf_counter()
print(f"train time {(e-s)*1000} ms")

s = time.perf_counter()
index.add(xb)
e =  time.perf_counter()
print(f"add time {(e-s)*1000} ms")

train time 0.0638999990769662 ms
add time 219.86944699892774 ms


In [59]:
import numpy as np
import time

xb.shape
nb, d = xb.shape

index = faiss.index_factory(d, "IVF1,Flat") 
s = time.perf_counter()
index.train(xb)
e =  time.perf_counter()
print(f"train time {(e-s)*1000} ms")

s = time.perf_counter()
index.add(xb)
e =  time.perf_counter()
print(f"add time {(e-s)*1000} ms")

train time 49.05525000503985 ms
add time 490.9680650016526 ms


In [45]:
import numpy as np
import time

xb.shape
nb, d = xb.shape

index = faiss.index_factory(d, "IVF100,Flat") 
s = time.perf_counter()
index.train(xb)
e =  time.perf_counter()
print(f"train time {(e-s)*1000} ms")

s = time.perf_counter()
index.add(xb)
e =  time.perf_counter()
print(f"add time {(e-s)*1000} ms")

train time 157.82671800116077 ms
add time 595.5642119952245 ms


In [46]:
import numpy as np
import time

xb.shape
nb, d = xb.shape

index = faiss.index_factory(d, "IVF200,Flat") 
s = time.perf_counter()
index.train(xb)
e =  time.perf_counter()
print(f"train time {(e-s)*1000} ms")

s = time.perf_counter()
index.add(xb)
e =  time.perf_counter()
print(f"add time {(e-s)*1000} ms")

train time 426.9496729975799 ms
add time 880.6673369981581 ms


In [63]:
import numpy as np
import time

xb.shape
nb, d = xb.shape

index = faiss.index_factory(d, "IVF1000,Flat") 
s = time.perf_counter()
index.train(xb)
e =  time.perf_counter()
print(f"train time {(e-s)*1000} ms")

s = time.perf_counter()
index.add(xb)
e =  time.perf_counter()
print(f"add time {(e-s)*1000} ms")

train time 8184.486572004971 ms
add time 3309.988193999743 ms


In [167]:
xb.shape

(1000000, 956)

In [62]:
nq = 100
index.nprobe = 1

# Function to count number of values not equal to -1 in the indices
def count_non_negative_indices(indices):
    return np.sum(indices != -1, axis=0)

### warm up run
# First query (k=10)
k = 100
s = time.perf_counter()
distances, indices = index.search(xq[:100, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

#####
# First query (k=10)
k = 1
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = 10
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = 100
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = 1000
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = nb//3
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = nb//2
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = nb
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)


query 100 time 2778.337627998553 ms
(avg)num returned: 10000
(100, 100)
query 1 time 2845.899668005586 ms
(avg)num returned: 100
(100, 1)
query 10 time 2865.438306995202 ms
(avg)num returned: 1000
(100, 10)
query 100 time 2752.43580499955 ms
(avg)num returned: 10000
(100, 100)
query 1000 time 2762.652632998652 ms
(avg)num returned: 100000
(100, 1000)
query 333333 time 7939.658374998544 ms
(avg)num returned: 33333300
(100, 333333)
query 500000 time 10694.775520001713 ms
(avg)num returned: 50000000
(100, 500000)
query 1000000 time 19190.429387999757 ms
(avg)num returned: 100000000
(100, 1000000)


In [65]:
nq = 100
index.nprobe = 25

# Function to count number of values not equal to -1 in the indices
def count_non_negative_indices(indices):
    return np.sum(indices != -1, axis=0)

### warm up run
# First query (k=10)
k = 100
s = time.perf_counter()
distances, indices = index.search(xq[:100, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

#####
# First query (k=10)
k = 1
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = 10
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = 100
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = 1000
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = nb//3
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = nb//2
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)

k = nb
s = time.perf_counter()
distances, indices = index.search(xq[:nq, :], k=k )
e = time.perf_counter()
print(f"query {k} time {(e-s)*1000} ms")
print(f"(avg)num returned: {count_non_negative_indices(indices.ravel()/ nq )}")
print(indices.shape)


query 100 time 50.503862999903504 ms
(avg)num returned: 10000
(100, 100)
query 1 time 45.54741500032833 ms
(avg)num returned: 100
(100, 1)
query 10 time 45.69512399757514 ms
(avg)num returned: 1000
(100, 10)
query 100 time 47.40501999913249 ms
(avg)num returned: 10000
(100, 100)
query 1000 time 53.863294000620954 ms
(avg)num returned: 100000
(100, 1000)
query 333333 time 1679.7387300030096 ms
(avg)num returned: 33333300
(100, 333333)
query 500000 time 2374.557087998255 ms
(avg)num returned: 50000000
(100, 500000)
query 1000000 time 4654.148317000363 ms
(avg)num returned: 100000000
(100, 1000000)


In [155]:
indices

array([[ 296,  646,  196, ...,   -1,   -1,   -1],
       [ 811, 1199,  965, ...,   -1,   -1,   -1],
       [  60, 1738, 1040, ...,   -1,   -1,   -1],
       ...,
       [ 390,  616,  244, ...,   -1,   -1,   -1],
       [ 344, 1243,  341, ...,   -1,   -1,   -1],
       [ 295,  767,  624, ...,   -1,   -1,   -1]])

In [50]:
indices

array([[715, 286,  11, ...,  -1,  -1,  -1]])

In [24]:
embeddings.shape

(1, 1, 16)

# Other

In [2]:
import faiss
from vasili_helpers import *

In [3]:
xb, xq, xt, gt, metric = load_sift1M("/pub/scratch/vmageirakos/datasets/ann-fvecs/sift-128-euclidean")

Loading sift1M...done


dataset shape:
xb.shape=(1000000, 128)
gt.shape=(10000, 100)
xq.shape=(10000, 128)


In [4]:
c_visited =  xb[:10,:]

In [5]:
local_quantizer = faiss.IndexFlat(128, faiss.METRIC_L2)
local_quantizer.add(c_visited)

In [6]:
local_quantizer.assign(xb,k=1).shape
local_quantizer.assign(xb,k=1).reshape(1,-1).shape

(1, 1000000)

In [7]:
local_quantizer.assign(xb,k=1).ravel()
local_quantizer.assign(xb,k=1).reshape(1,-1).ravel()

array([0, 1, 2, ..., 8, 6, 5])

Check direct map 

In [8]:
from CrackIVF import *

In [9]:
index = CrackIVF(nlist=1000, niter=10, seed=42)

In [10]:
index.add(xb)

Init Index... self.nlist=1000 self.niter=10 self.max_pts=256 self.seed=42 self.d=128
Training level-1 quantizer
Training level-1 quantizer on 1000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
IndexIVFFlat::add_core: added 1000000 / 1000000 vectors
	---> Index Train Time = 4828.998528002558 ms | Add Time = 1538.697903000866 ms <---


(4.828998528002558, 1.538697903000866)

In [11]:
index.index.direct_map.get(0)

1782411427840

In [12]:
dir(index.index)

['PARALLEL_MODE_NO_HEAP_INIT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'add',
 'add_c',
 'add_core',
 'add_sa_codes',
 'add_sa_codes_c',
 'add_with_ids',
 'add_with_ids_c',
 'assign',
 'assign_c',
 'by_residual',
 'check_compatible_for_merge',
 'check_ids_sorted',
 'clustering_index',
 'coarse_code_size',
 'code_size',
 'compute_residual',
 'compute_residual_n',
 'copy_subset_to',
 'cp',
 'd',
 'decode_listno',
 'direct_map',
 'encode_listno',
 'encode_vectors',
 'get_CodePacker',
 'get_InvertedListScanner',
 'get_distance_computer',
 'get_list_size',
 'invlists',
 'is_trained',
 'make_direct_map',
 'max_co

In [15]:
index.index.ntotal
index.index.nlist

1000000

1000

In [17]:
# dir(index.index)

In [23]:
index.index.add_with_ids(xb[0:1,:], 1000000)
# index.index.remove_ids(np.array([1000000]))

IndexIVFFlat::add_core: added 1 / 1 vectors


In [24]:
index.index.direct_map.get(999999+1)
index.index.direct_map.get(0)

1782411428619

1782411427840

In [55]:
# index.index.add_with_ids(xb[0:1,:], 1000000)

index.index.remove_ids(np.array([1000000]))

1

In [70]:
index.index.direct_map.get(0)
index.index.direct_map.get(999999+1)

1782411427840

1782411428620

In [69]:
add_preassigned(index.index, xb[0:1,:], np.array([415]).ravel(), ids=np.array([1000000]))

IndexIVFFlat::add_core: added 1 / 1 vectors


In [54]:
index.index.quantizer.assign(xb[:1,:],k=1)

array([[415]])

All these need to match to assert that add_preassigned() works with ids 

In [65]:
xb[0:1,:]

array([[  0.,  16.,  35.,   5.,  32.,  31.,  14.,  10.,  11.,  78.,  55.,
         10.,  45.,  83.,  11.,   6.,  14.,  57., 102.,  75.,  20.,   8.,
          3.,   5.,  67.,  17.,  19.,  26.,   5.,   0.,   1.,  22.,  60.,
         26.,   7.,   1.,  18.,  22.,  84.,  53.,  85., 119., 119.,   4.,
         24.,  18.,   7.,   7.,   1.,  81., 106., 102.,  72.,  30.,   6.,
          0.,   9.,   1.,   9., 119.,  72.,   1.,   4.,  33., 119.,  29.,
          6.,   1.,   0.,   1.,  14.,  52., 119.,  30.,   3.,   0.,   0.,
         55.,  92., 111.,   2.,   5.,   4.,   9.,  22.,  89.,  96.,  14.,
          1.,   0.,   1.,  82.,  59.,  16.,  20.,   5.,  25.,  14.,  11.,
          4.,   0.,   0.,   1.,  26.,  47.,  23.,   4.,   0.,   0.,   4.,
         38.,  83.,  30.,  14.,   9.,   4.,   9.,  17.,  23.,  41.,   0.,
          0.,   2.,   8.,  19.,  25.,  23.,   1.]], dtype=float32)

In [64]:
index.index.reconstruct(0)

array([  0.,  16.,  35.,   5.,  32.,  31.,  14.,  10.,  11.,  78.,  55.,
        10.,  45.,  83.,  11.,   6.,  14.,  57., 102.,  75.,  20.,   8.,
         3.,   5.,  67.,  17.,  19.,  26.,   5.,   0.,   1.,  22.,  60.,
        26.,   7.,   1.,  18.,  22.,  84.,  53.,  85., 119., 119.,   4.,
        24.,  18.,   7.,   7.,   1.,  81., 106., 102.,  72.,  30.,   6.,
         0.,   9.,   1.,   9., 119.,  72.,   1.,   4.,  33., 119.,  29.,
         6.,   1.,   0.,   1.,  14.,  52., 119.,  30.,   3.,   0.,   0.,
        55.,  92., 111.,   2.,   5.,   4.,   9.,  22.,  89.,  96.,  14.,
         1.,   0.,   1.,  82.,  59.,  16.,  20.,   5.,  25.,  14.,  11.,
         4.,   0.,   0.,   1.,  26.,  47.,  23.,   4.,   0.,   0.,   4.,
        38.,  83.,  30.,  14.,   9.,   4.,   9.,  17.,  23.,  41.,   0.,
         0.,   2.,   8.,  19.,  25.,  23.,   1.], dtype=float32)

In [71]:
index.index.reconstruct(1000000)

array([  0.,  16.,  35.,   5.,  32.,  31.,  14.,  10.,  11.,  78.,  55.,
        10.,  45.,  83.,  11.,   6.,  14.,  57., 102.,  75.,  20.,   8.,
         3.,   5.,  67.,  17.,  19.,  26.,   5.,   0.,   1.,  22.,  60.,
        26.,   7.,   1.,  18.,  22.,  84.,  53.,  85., 119., 119.,   4.,
        24.,  18.,   7.,   7.,   1.,  81., 106., 102.,  72.,  30.,   6.,
         0.,   9.,   1.,   9., 119.,  72.,   1.,   4.,  33., 119.,  29.,
         6.,   1.,   0.,   1.,  14.,  52., 119.,  30.,   3.,   0.,   0.,
        55.,  92., 111.,   2.,   5.,   4.,   9.,  22.,  89.,  96.,  14.,
         1.,   0.,   1.,  82.,  59.,  16.,  20.,   5.,  25.,  14.,  11.,
         4.,   0.,   0.,   1.,  26.,  47.,  23.,   4.,   0.,   0.,   4.,
        38.,  83.,  30.,  14.,   9.,   4.,   9.,  17.,  23.,  41.,   0.,
         0.,   2.,   8.,  19.,  25.,  23.,   1.], dtype=float32)